In [ ]:
import os, pandas as pd, numpy as np, librosa, random, pygame, time, threading, queue
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Configuration
CACHE_FILE = 'music_analysis_cache.csv'
NUM_CLUSTERS = 5
SUPPORTED_EXTENSIONS = ['.mp3', '.m4a', '.flac', '.wav']
MOOD_NAMES = ["Deep Focus & Ambient", "Acoustic & Calm", "Groovy & Rhythmic", "Bright & Pop", "Workout & Energy"]

class ISAI(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("ISAI - Intelligent Music Player")
        self.geometry("600x450")
        self.configure(bg='#2E2E2E')
        
        # Initialize variables
        self.music_folder = ""
        self.df = None
        self.mood_map = None
        self.player = None
        self.is_paused = False
        self.progress_queue = queue.Queue()
        
        # Setup GUI
        self.setup_gui()
        self.after(100, self.process_queue)
    
    def setup_gui(self):
        # Configure styles
        style = ttk.Style(self)
        style.theme_use('clam')
        style.configure('TFrame', background='#2E2E2E')
        style.configure('TLabel', background='#2E2E2E', foreground='white', font=('Helvetica', 10))
        style.configure('TButton', font=('Helvetica', 10, 'bold'))
        style.map('TButton', background=[('active', '#4A4A4A')])
        style.configure('TCombobox', fieldbackground='#3E3E3E', background='#3E3E3E', foreground='white')
        style.configure('Horizontal.TProgressbar', background='#4CAF50')
        
        # Main container
        main_frame = ttk.Frame(self, padding="10")
        main_frame.pack(fill=tk.BOTH, expand=True)
        
        # Setup frame
        self.setup_frame = ttk.Frame(main_frame)
        self.setup_frame.pack(fill=tk.BOTH, expand=True)
        ttk.Label(self.setup_frame, text="Welcome to ISAI!", font=('Helvetica', 16)).pack(pady=20)
        self.select_button = ttk.Button(self.setup_frame, text="Select Music Folder", command=self.select_folder)
        self.select_button.pack(pady=10)
        self.status_label = ttk.Label(self.setup_frame, text="Please select a folder to begin.")
        self.status_label.pack(pady=5)
        self.progress_bar = ttk.Progressbar(self.setup_frame, orient="horizontal", length=300, mode="determinate")
        self.progress_bar.pack(pady=10, padx=20, fill=tk.X)
        
        # Player frame (initially hidden)
        self.player_frame = ttk.Frame(main_frame)
        
        # Mood selection
        mood_frame = ttk.Frame(self.player_frame)
        mood_frame.pack(fill=tk.X, pady=5)
        ttk.Label(mood_frame, text="Select Mood:").pack(side=tk.LEFT, padx=5)
        self.mood_var = tk.StringVar()
        self.mood_combobox = ttk.Combobox(mood_frame, textvariable=self.mood_var, state="readonly")
        self.mood_combobox.pack(side=tk.LEFT, fill=tk.X, expand=True, padx=5)
        self.mood_combobox.bind("<<ComboboxSelected>>", self.on_mood_selected)
        
        # Now playing
        self.now_playing_label = ttk.Label(self.player_frame, text="No song playing", font=('Helvetica', 12, 'italic'))
        self.now_playing_label.pack(pady=10)
        
        # Song progress
        self.song_progress_bar = ttk.Progressbar(self.player_frame, orient="horizontal", length=400, mode="determinate")
        self.song_progress_bar.pack(pady=5, padx=20, fill=tk.X)
        
        # Control buttons
        controls_frame = ttk.Frame(self.player_frame)
        controls_frame.pack(pady=10)
        ttk.Button(controls_frame, text="⏮", command=self.play_previous, width=3).pack(side=tk.LEFT, padx=5)
        self.play_pause_button = ttk.Button(controls_frame, text="▶", command=self.toggle_play_pause, width=3)
        self.play_pause_button.pack(side=tk.LEFT, padx=5)
        ttk.Button(controls_frame, text="⏭", command=self.play_next, width=3).pack(side=tk.LEFT, padx=5)
        ttk.Button(controls_frame, text="⏪", command=lambda: self.seek(-10), width=3).pack(side=tk.LEFT, padx=5)
        ttk.Button(controls_frame, text="⏩", command=lambda: self.seek(10), width=3).pack(side=tk.LEFT, padx=5)
    
    def select_folder(self):
        folder = filedialog.askdirectory()
        if folder:
            self.music_folder = folder
            self.status_label.config(text="Starting analysis...")
            self.progress_bar.config(value=0, maximum=0)
            self.select_button.config(state="disabled")
            threading.Thread(target=self.analyze_music, daemon=True).start()
    
    def analyze_music(self):
        try:
            # --- ROBUST CACHE LOADING ---
            if os.path.exists(CACHE_FILE):
                self.progress_queue.put(('status', "Loading from cache..."))
                try:
                    # Load CSV as a simple table, filepath is a regular column
                    df = pd.read_csv(CACHE_FILE)
                    required_columns = ['filepath', 'tempo', 'cluster', 'spectral_centroid', 'zero_crossing_rate', 'spectral_rolloff']
                    if all(col in df.columns for col in required_columns):
                        # Set the index in memory, making it ready for the app
                        df.set_index('filepath', inplace=True)
                        self.df = df
                        self.progress_queue.put(('done', None))
                        return
                except Exception as e:
                    # If cache is corrupt, delete it and re-analyze
                    print(f"Cache is corrupted or invalid. Re-analyzing. Error: {e}")
                    try:
                        os.remove(CACHE_FILE)
                    except OSError:
                        pass
            
            # --- FIND MUSIC FILES ---
            self.progress_queue.put(('status', "Finding music files..."))
            music_files = []
            for root, _, files in os.walk(self.music_folder):
                for file in files:
                    if any(file.lower().endswith(ext) for ext in SUPPORTED_EXTENSIONS):
                        music_files.append(os.path.join(root, file))
            
            if not music_files:
                self.progress_queue.put(('error', "No supported music files found."))
                return
            
            # --- EXTRACT FEATURES ---
            self.progress_queue.put(('status', "Analyzing music files..."))
            self.progress_queue.put(('progress_max', len(music_files)))
            
            all_features = []
            for file_path in music_files:
                try:
                    y, sr = librosa.load(file_path, duration=60)
                    features = {'filepath': os.path.abspath(file_path)}
                    features['tempo'], _ = librosa.beat.beat_track(y=y, sr=sr)
                    features['spectral_centroid'] = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr))
                    features['spectral_rolloff'] = np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr))
                    features['zero_crossing_rate'] = np.mean(librosa.feature.zero_crossing_rate(y))
                    mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13), axis=1)
                    for i, mfcc in enumerate(mfccs):
                        features[f'mfcc_{i+1}'] = mfcc
                    all_features.append(features)
                    self.progress_queue.put(('progress', 1))
                except Exception as e:
                    print(f"Error processing {os.path.basename(file_path)}: {e}")
                    self.progress_queue.put(('progress', 1))
            
            if not all_features:
                self.progress_queue.put(('error', "Could not process any music files."))
                return
            
            # --- CLUSTER THE MUSIC ---
            df = pd.DataFrame(all_features)
            df.set_index('filepath', inplace=True)
            scaler = StandardScaler()
            X_scaled = scaler.fit_transform(df)
            kmeans = KMeans(n_clusters=NUM_CLUSTERS, random_state=42, n_init=10)
            df['cluster'] = kmeans.fit_predict(X_scaled)
            
            # --- ROBUST CACHE SAVING ---
            df_to_save = df.reset_index() # Turn index back into a column
            df_to_save.to_csv(CACHE_FILE, index=False) # Save without pandas index
            
            self.df = df # Keep the indexed version for the app
            self.progress_queue.put(('done', None))
            
        except Exception as e:
            self.progress_queue.put(('error', f"An error occurred: {e}"))
    
    def process_queue(self):
        try:
            while True:
                msg = self.progress_queue.get_nowait()
                if msg[0] == 'progress':
                    self.progress_bar.step(msg[1])
                elif msg[0] == 'progress_max':
                    self.progress_bar.config(maximum=msg[1], value=0)
                elif msg[0] == 'status':
                    self.status_label.config(text=msg[1])
                elif msg[0] == 'done':
                    self.status_label.config(text="Analysis complete!")
                    self.progress_bar.config(value=100)
                    self.show_player()
                    return
                elif msg[0] == 'error':
                    messagebox.showerror("Error", msg[1])
                    self.reset_ui()
                    return
        except queue.Empty:
            pass
        self.after(100, self.process_queue)
    
    def show_player(self):
        self.setup_frame.pack_forget()
        self.player_frame.pack(fill=tk.BOTH, expand=True)
        
        # Create mood map
        df = self.df.copy()
        df['tempo'] = pd.to_numeric(df['tempo'], errors='coerce')
        numeric_cols = df.select_dtypes(include=np.number).columns.drop('cluster')
        cluster_means = df.groupby('cluster')[numeric_cols].mean()
        sorted_clusters = cluster_means.sort_values(by='tempo', ascending=True)
        self.mood_map = {cluster_id: MOOD_NAMES[i] for i, cluster_id in enumerate(sorted_clusters.index)}
        
        # Setup mood combobox
        self.mood_combobox['values'] = list(self.mood_map.values())
        if self.mood_combobox['values']:
            self.mood_combobox.current(0)
            self.on_mood_selected(None)
    
    def on_mood_selected(self, event):
        if not self.mood_var.get() or self.df is None:
            return
        
        selected_mood = self.mood_var.get()
        target_cluster = next(k for k, v in self.mood_map.items() if v == selected_mood)
        
        songs_in_mood = self.df[self.df['cluster'] == target_cluster]
        playlist_paths = songs_in_mood.index.tolist()
        random.shuffle(playlist_paths)
        
        if self.player:
            self.player.stop()
        
        self.player = MusicPlayer(playlist_paths, self)
        self.player.play_current_song()
        self.update_song_progress()
    
    def toggle_play_pause(self):
        if self.player:
            self.player.toggle_play_pause()
            self.play_pause_button.config(text="▶" if self.is_paused else "⏸")
    
    def play_next(self):
        if self.player:
            self.player.play_next()
    
    def play_previous(self):
        if self.player:
            self.player.play_previous()
    
    def seek(self, seconds):
        if self.player:
            if seconds < 0:
                self.player.rewind(abs(seconds))
            else:
                self.player.forward(seconds)
    
    def update_now_playing(self, song_name):
        self.now_playing_label.config(text=f"Now Playing: {song_name}")
    
    def update_song_progress(self):
        if self.player and self.player.playlist:
            if not pygame.mixer.music.get_busy() and not self.is_paused:
                self.player.play_next()
            
            try:
                current_pos = self.player.get_current_position()
                duration = self.player.song_duration
                if duration > 0:
                    progress = (current_pos / duration) * 100
                    self.song_progress_bar['value'] = progress
            except:
                pass
        
        self.after(500, self.update_song_progress)
    
    def reset_ui(self):
        self.select_button.config(state="normal")
        self.progress_bar.config(value=0, maximum=100)
        self.status_label.config(text="Please select a folder to begin.")

class MusicPlayer:
    def __init__(self, playlist, gui_app):
        self.playlist = playlist
        self.current_index = 0
        self.gui_app = gui_app
        self.is_paused = False
        self.current_position = 0
        self.song_duration = 0
        self.last_seek_time = 0
        
        if not self.playlist:
            print("Playlist is empty.")
            return
        try:
            pygame.mixer.init()
        except pygame.error as e:
            print(f"Pygame mixer error: {e}")
            self.playlist = []
    
    def play_current_song(self):
        if not self.playlist: 
            return
        filepath = self.playlist[self.current_index]
        try:
            self.song_duration = librosa.get_duration(path=filepath)
            pygame.mixer.music.load(filepath)
            pygame.mixer.music.play()
            self.is_paused = False
            self.current_position = 0
            self.last_seek_time = time.time()
            self.gui_app.update_now_playing(os.path.basename(filepath))
            self.gui_app.is_paused = False
            self.gui_app.play_pause_button.config(text="⏸")
        except Exception as e:
            print(f"Error playing {os.path.basename(filepath)}: {e}")
    
    def toggle_play_pause(self):
        if not self.playlist: 
            return
        if pygame.mixer.music.get_busy():
            if self.is_paused:
                pygame.mixer.music.unpause()
                self.is_paused = False
                self.last_seek_time = time.time()
                self.gui_app.is_paused = False
            else:
                pygame.mixer.music.pause()
                self.is_paused = True
                self.gui_app.is_paused = True
        else:
            self.play_current_song()
    
    def play_next(self):
        if not self.playlist: 
            return
        self.current_index = (self.current_index + 1) % len(self.playlist)
        self.play_current_song()
    
    def play_previous(self):
        if not self.playlist: 
            return
        self.current_index = (self.current_index - 1 + len(self.playlist)) % len(self.playlist)
        self.play_current_song()
    
    def get_current_position(self):
        if not pygame.mixer.music.get_busy() or self.is_paused:
            return self.current_position
        elapsed = time.time() - self.last_seek_time
        return min(self.current_position + elapsed, self.song_duration)
    
    def rewind(self, seconds=10):
        if not self.playlist: 
            return
        current_pos = self.get_current_position()
        new_pos = max(0, current_pos - seconds)
        self.seek_to(new_pos)
    
    def forward(self, seconds=10):
        if not self.playlist: 
            return
        current_pos = self.get_current_position()
        new_pos = min(current_pos + seconds, self.song_duration)
        self.seek_to(new_pos)
    
    def seek_to(self, position_seconds):
        was_paused = self.is_paused
        filepath = self.playlist[self.current_index]
        pygame.mixer.music.stop()
        pygame.mixer.music.load(filepath)
        pygame.mixer.music.play(start=position_seconds)
        self.current_position = position_seconds
        self.last_seek_time = time.time()
        if was_paused:
            pygame.mixer.music.pause()
            self.is_paused = True
            self.gui_app.is_paused = True
    
    def stop(self):
        pygame.mixer.music.stop()
        pygame.mixer.quit()

if __name__ == "__main__":
    app = ISAI()
    app.mainloop()